## 实验1-1 循环授信贷款产品的客户违约率  

基于客户数据 (data_client_1.xlsx) 和借贷历史（借据）数据 (data_client_1_hist.xlsx)，计算：  

1）**通过率**：审批/授信通过率  

2）**转化率**：授信后的借贷比例  

3）**授信⾦额**：授信金额总和  

4）**客户违约率**：借贷天数超过90天且仍未还款的客户比例  


### 关于文件目录  

**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  

**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  

**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [21]:
import numpy as np
import pandas as pd

In [22]:
# 读取两份EXCEL文件
df1 = pd.read_excel("data_client_1.xlsx")
df2 = pd.read_excel("data_client_1_hist.xlsx")

In [23]:
# 计算通过率
approval_rate = df1['if_approved'].sum() / len(df1['if_approved'])
# 计算转化率
transfer_rate = df1['if_used'].sum() / df1['if_approved'].sum()
# 计算授信金额
credit_sum = df1['credit_approved'].sum()
# 计算客户违约率
default_clients = df2.loc[df2.loan_repaid_date.isna() & (df2.loan_used_days > 90), 'client_no'].nunique()
default_rate = default_clients / df1['if_used'].sum()

[approval_rate, transfer_rate, credit_sum, default_rate]

[0.5904, 0.34010840108401086, 81550955, 0.10557768924302789]

## 实验1-2 循环授信贷款中不同人群的风险成本  
基于客户数据 (data_client_1.xlsx)和借贷历史(借据)数据 (data_client_1_hist.xlsx)，计算：  
**不同人群间的违约率、违约损失率、风险敞口、预期损失及实际损失**。  

将客户按credit_score划分为两个客户群：  

1）160~190：160<=credit_score<=190  

2）190+：credit_score>190  

预期损失的计算公式：  

**Expected Loss= Probability of Default * Loss Given Default * Exposure at Default**  

Probability of Default (PD)：违约率  

Loss Given Default (LGD)：违约损失率  

Exposure at Default (EaD)：⻛险敞⼝  

**注意：**  

* 上述违约率与1中的“客户违约率”不同，指的是“借据违约率”  

* 计算时将逾期大于110天仍未还款的借据看作损失  

* 在估计产品的风险成本时，EaD计算的是贷款总金额  

* 损失、敞口都和金额相关的  

In [24]:
# 计算违约率
def cal_pd(df):
    # 违约(逾期）数
    a = len(df.loc[df.loan_used_days > 90, 'client_no'])
    # 借据数
    b = len(df.loc[df.loan_order > 0, 'client_no'])

    return a/b

In [25]:
# 计算违约损失率
def cal_lgd(df):
    # 损失金额
    loss = df.loc[df.loan_used_days > 110, 'loan_amt'].sum()
    # 违约金额
    default = df.loc[df.loan_used_days > 90, 'loan_amt'].sum()

    return loss/default


In [26]:
# 计算风险敞口
def cal_ead(df):
    return df.loan_amt.sum()

In [27]:
# 计算实际损失
def cal_loss(df):
    return df.loc[df.loan_used_days > 110, 'loan_amt'].sum()

In [28]:
group1 = df1.loc[(df1.credit_score >= 160) & (df1.credit_score <= 190), 'client_no']
group2 = df1.loc[df1.credit_score > 190, 'client_no']

df2_group1 = df2.loc[df2.client_no.isin(group1)]
df2_group2 = df2.loc[df2.client_no.isin(group2)]

# 计算客群1的违约率、违约损失率、风险敞口、预期损失及实际损失
pd1 = cal_pd(df2_group1)
lgd1 = cal_lgd(df2_group1)
ead1 = cal_ead(df2_group1)
expect_loss1 = cal_ead(df2_group1)
exact_loss1 = cal_loss(df2_group1)

# 计算客群2的违约率、违约损失率、风险敞口、预期损失及实际损失
pd2 = cal_pd(df2_group2)
lgd2 = cal_lgd(df2_group2)
ead2 = cal_ead(df2_group2)
expect_loss2 = cal_ead(df2_group2)
exact_loss2 = cal_loss(df2_group2)

res = {'160~190': {
        'pd': pd1,
        'lgd': lgd1,
        'ead': ead1,
        'expected_loss': expect_loss1,
        'exact_loss': exact_loss1}
}

res['190+'] = {
        'pd': pd2,
        'lgd': lgd2,
        'ead': ead2,
        'expected_loss': expect_loss2,
        'exact_loss': exact_loss2}

res

{'160~190': {'pd': 0.7687264506405426,
  'lgd': 0.0001986628337110357,
  'ead': 294033694.24129575,
  'expected_loss': 294033694.24129575,
  'exact_loss': 46000.0},
 '190+': {'pd': 0.008771929824561403,
  'lgd': 0.0,
  'ead': 7187311.926664613,
  'expected_loss': 7187311.926664613,
  'exact_loss': 0.0}}